# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load("en_core_web_sm")

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(" "))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open("ner-train.tsv.bz2", 'rt') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open("ner-dev.tsv.bz2", 'rt') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    intersection = list(gold & pred)
    
    precision = len(intersection)/len(pred)
    recall = len(intersection)/len(gold)
    F1 = 2*(precision*recall)/(precision+recall)
    #return {"precision": precision, "recall": recall, "f1": F1}
    print("precision:", precision, "\nrecall:   ", recall, "\nf1-score: ", F1)

To test your code, you can run the following cell:

In [6]:
print(set(range(3)))
print(set(range(5)))

{0, 1, 2}
{0, 1, 2, 3, 4}


In [7]:
evaluation_report(set(range(3)), set(range(5)))

precision: 0.6 
recall:    1.0 
f1-score:  0.7499999999999999


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [8]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    for row in df.iloc[:,[0,2,3]].itertuples(index=False, name=None):
        yield row

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [9]:
gen = list(gold_spans(df_dev))
spans_dev_gold = set(gen)
print(len(spans_dev_gold))

5917


In [10]:
print("The first triple is",gen[0],"\nThe last triple is",gen[-1])

The first triple is ('0946-000', 2, 3) 
The last triple is ('1161-010', 1, 3)


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [11]:
def spacyEntities(df):
    res=[]
    for row in df.iloc[:, [0,1]].drop_duplicates().itertuples(index=False):
        doc = nlp(row[1])
        for ent in doc.ents:
            res.append((row[0], ent.start, ent.end))
    return(res)

In [12]:
entities = spacyEntities(df_dev)

In [13]:
evaluation_report(set(gold_spans(df_dev)),set(entities))

precision: 0.5153023140084598 
recall:    0.7000169004563124 
f1-score:  0.5936223575779291


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Write code that prints the false positives and the false negatives from the automatic prediction. Have a look at the output. What are your observations? How could you improve the result? Discuss these questions in a short text.

In [14]:
FN = list(set(gold_spans(df_dev)) - set(entities))
FP = list(set(entities) - set(gold_spans(df_dev)))

In [15]:
print("FN count:", len(FN))
print("Fp count:", len(FP))

FN count: 1775
Fp count: 3896


In [16]:
%%time
def get_entity_labels(entities, df):
    res = []
    for s_id, start, end in entities:
        entity_ = " ".join(df.loc[df_dev.sentence_id==s_id, "sentence"].values[0].split()[start:end])
        ents = nlp(entity_).ents
        entity = ents[0].label_ if len(ents)!=0 else "None"
        res.append(entity)
    return(res)
        
FP_entity_labels = get_entity_labels(FP, df_dev)
FN_entity_labels = get_entity_labels(FN, df_dev)
gold_labels = get_entity_labels(gold_spans(df_dev), df_dev)

CPU times: user 1min 6s, sys: 328 ms, total: 1min 7s
Wall time: 1min 7s


In [17]:
print("FP label counts:")
print(pd.DataFrame({"label": FP_entity_labels}).label.value_counts())
print("\nFN label counts:")
print(pd.DataFrame({"label": FN_entity_labels}).label.value_counts())
print("\nGolden label counts:")
print(pd.DataFrame({"label": gold_labels}).label.value_counts())

FP label counts:
CARDINAL       1923
DATE            781
ORG             226
ORDINAL         219
None            209
PERSON          134
GPE              88
TIME             82
QUANTITY         55
MONEY            53
PERCENT          46
NORP             35
EVENT            25
LAW              11
LOC               4
WORK_OF_ART       2
FAC               2
PRODUCT           1
Name: label, dtype: int64

FN label counts:
None        1015
PERSON       281
ORG          245
GPE          158
NORP          38
DATE          12
EVENT         11
CARDINAL       8
PRODUCT        2
ORDINAL        2
LOC            2
FAC            1
Name: label, dtype: int64

Golden label counts:
None        1899
GPE         1445
PERSON      1257
ORG          736
NORP         490
EVENT         28
DATE          17
ORDINAL       16
CARDINAL      16
LOC            6
FAC            5
PRODUCT        2
Name: label, dtype: int64


**Analysis:**

We can see that most of the labels in golden entities, we have some outliers from `EVENT` to end of list. Also, we can see these outliers in our FN and FP labels as a big proportion. So if we can filter those outlier labels in the prediction phase, we can achieve a better F1 score by removing the major cause of having False Positive and False Negative errors.  

We will filter out the predictions for the labels `["CARDINAL", "DATE", "ORDINAL", "TIME", "QUANTITY", "MONEY", "PERCENT"]`

We selected the labels by checking the majority labels in FN and FP if they are minority in the Golden labels.

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to pre-process the data and/or post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [18]:
def spacyEntities(df, filter_labels = []):
    res=[]
    for row in df.iloc[:, [0,1]].drop_duplicates().itertuples(index=False):
        doc = nlp(row[1])
        for ent in doc.ents:
            if not ent.label_ in filter_labels:
                res.append((row[0], ent.start, ent.end))
    return(res)

In [19]:
%%time
filter_labels = ["CARDINAL", "DATE", "ORDINAL", "TIME", "QUANTITY", "MONEY", "PERCENT"]
entities = spacyEntities(df_dev, filter_labels)

CPU times: user 16.1 s, sys: 99.8 ms, total: 16.2 s
Wall time: 16.3 s


In [20]:
evaluation_report(set(gold_spans(df_dev)),set(entities))

precision: 0.86005406529424 
recall:    0.6990028730775731 
f1-score:  0.7712101435763565


Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the predicted start and end positions for each token span. As the `label` of each span, you can use the special value `--NME--`.

In [21]:
df_dev.head()

,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,2,3,Leicestershire_County_Cricket_Club
1,0946-001,LONDON 1996-08-30,0,1,London
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indies_cricket_team
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire_County_Cricket_Club


In [22]:
%%time
df_entities = pd.DataFrame(entities, columns=["sentence_id", "beg", "end"])
df_entities["sentence"] = df_entities.sentence_id.apply(lambda x: df_dev.loc[df_dev.sentence_id==x].sentence.values[0])
df_entities["label"] = ["--NME--"]* df_entities.shape[0]
df_entities = df_entities.loc[:, ["sentence_id", "sentence", "beg", "end", "label"]]
df_entities

CPU times: user 4.08 s, sys: 16 ms, total: 4.1 s
Wall time: 4.12 s


,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,0,3,--NME--
1,0946-001,LONDON 1996-08-30,0,1,--NME--
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,--NME--
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,--NME--
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,--NME--
...,...,...,...,...,...
4804,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",7,9,--NME--
4805,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",10,12,--NME--
4806,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",13,15,--NME--
4807,1161-009,The DSE all share price index closed 2.73 poin...,1,2,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [23]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    for row in df.itertuples():
        yield (row[1], row[3], row[4], row[5])

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [24]:
def naive_baseline(df):
    df.label = df.apply(lambda row: "_".join(row.sentence.split()[row.beg:row.end]), axis=1)
    return(df)

df_naive = naive_baseline(df_entities)
df_naive

,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,0,3,CRICKET_-_LEICESTERSHIRE
1,0946-001,LONDON 1996-08-30,0,1,LONDON
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indian
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire
...,...,...,...,...,...
4804,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",7,9,Bangladesh_Lamps
4805,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",10,12,Chittagong_Cement
4806,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",13,15,Atlas_Bangladesh
4807,1161-009,The DSE all share price index closed 2.73 poin...,1,2,DSE


In [25]:
evaluation_report(set(gold_mentions(df_dev)), set(gold_mentions(df_naive)))

precision: 0.3160740278644209 
recall:    0.2568869359472706 
f1-score:  0.28342345702032445


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on when you click on a link) as mentions of the entities (pages) that they link to. This allows us to harvest long lists over mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [26]:
with bz2.open("kb.tsv.bz2", 'rt') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [27]:
df_kb.loc[df_kb["mention"] == "Sweden"]

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing stands first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [28]:
df_entities

,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,0,3,CRICKET_-_LEICESTERSHIRE
1,0946-001,LONDON 1996-08-30,0,1,LONDON
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indian
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire
...,...,...,...,...,...
4804,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",7,9,Bangladesh_Lamps
4805,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",10,12,Chittagong_Cement
4806,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",13,15,Atlas_Bangladesh
4807,1161-009,The DSE all share price index closed 2.73 poin...,1,2,DSE


In [29]:
def most_probable_entity(df_en, df_kb):
    labels = []
    for row in df_en.itertuples(index=False):
        mention = " ".join(row.sentence.split()[row.beg:row.end])
        try:
            labels.append(df_kb.loc[df_kb.loc[df_kb["mention"] == mention, "prob"].idxmax(), "entity"])
        except:
            labels.append("--NME--")
    return labels

In [30]:
labels = most_probable_entity(df_entities, df_kb)
df_entities.label = labels
df_entities

,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,0,3,--NME--
1,0946-001,LONDON 1996-08-30,0,1,London
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indies_cricket_team
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire_County_Cricket_Club
...,...,...,...,...,...
4804,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",7,9,--NME--
4805,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",10,12,--NME--
4806,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",13,15,--NME--
4807,1161-009,The DSE all share price index closed 2.73 poin...,1,2,Dhaka_Stock_Exchange


In [31]:
evaluation_report(set(gold_mentions(df_dev)), set(gold_mentions(df_entities)))

precision: 0.6598045331669786 
recall:    0.5362514787899273 
f1-score:  0.5916464665299274


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [32]:
with bz2.open("contexts.tsv.bz2") as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as a bag of words containing the mention itself, as well as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [33]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in 1970 and the second in ...
1,1970,FIFA_World_Cup,America 1975 and during the 1970 and 1978 Worl...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the 1990 World Cup
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the 1990 W...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the 1990 World Cup The...


From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [34]:
df_contexts.context[df_contexts.mention == "Lincoln"]

41465    Nebraska Concealed Handgun Permit In Lincoln m...
41466    Lazlo restaurants are located in Lincoln and O...
41467    California Washington Overland Park Kansas Lin...
41468    City Missouri Omaha Nebraska and Lincoln Nebra...
41469    by Sandhills Publishing Company in Lincoln Neb...
                               ...                        
41609                                Lincoln Leyton Orient
41610               English division three Swansea Lincoln
41611    league membership narrowly edging out Lincoln ...
41612                                    Lincoln Cambridge
41613                                              Lincoln
Name: context, Length: 149, dtype: object

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** To tune the performance of your method, you can try to tweak the behaviour of the vectorizer (for example, should it apply lowercasing or not?) and the width of the window from which you are extracting context tokens at prediction time.

**Hint 3:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [35]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [36]:
mentions = df_contexts.mention.unique()
nb_mentions = {}

In [37]:
%%time
# Train NB for each mention
for m in mentions:
    docs = df_contexts.loc[df_contexts.mention==m, "context"]
    entities = df_contexts.loc[df_contexts.mention==m, "entity"]
    prior = df_kb.loc[df_kb.mention==m].sort_values(by="entity").prob.values
    pipeline = make_pipeline(CountVectorizer(),
                            MultinomialNB(class_prior=prior))
    pipeline.fit(docs, entities)
    nb_mentions[m] = pipeline

CPU times: user 15.9 s, sys: 92.2 ms, total: 16 s
Wall time: 16.1 s


In [38]:
def naive_prediction(df_en, df_kb, nb_mentions):
    labels = []
    for row in df_en.itertuples(index=False):
        mention = " ".join(row.sentence.split()[row.beg:row.end])
        if mention in nb_mentions.keys():
            labels.append(nb_mentions[mention].predict([row.sentence])[0])
        elif mention in df_kb.mention.values:
            labels.append(df_kb.loc[df_kb.loc[df_kb["mention"] == mention, "prob"].idxmax(), "entity"])
        else:
            labels.append("--NME--")
    return(labels)

In [39]:
%%time
labels = naive_prediction(df_entities, df_kb, nb_mentions)
df_entities.label = labels
df_entities

CPU times: user 4.16 s, sys: 17.5 ms, total: 4.18 s
Wall time: 4.2 s


,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,0,3,--NME--
1,0946-001,LONDON 1996-08-30,0,1,London
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,Caribbean
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire_County_Cricket_Club
...,...,...,...,...,...
4804,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",7,9,--NME--
4805,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",10,12,--NME--
4806,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",13,15,--NME--
4807,1161-009,The DSE all share price index closed 2.73 poin...,1,2,Dhaka_Stock_Exchange


In [40]:
evaluation_report(set(gold_mentions(df_dev)), set(gold_mentions(df_entities)))

precision: 0.6776876689540445 
recall:    0.5507858712185228 
f1-score:  0.6076822673876562


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1. Published systems report a larger impact of context-sensitive disambiguation. Feel free to think about what could explain the relatively minor impact that we see here!

**This was the last lab in the Text Mining course. Well done!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>